# IF702 Redes Neurais
Esse notebook contém um script base para o projeto da disciplina IF702 Redes Neurais.

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

## Leitura e Limpeza dos Dados

A leitura do data set é feita utilizando a biblioteca `pandas`. O presente exemplo importa a base de dados `mammography`, assim, caso você esteja trabalhando com outro data set, modifique esta linha.
Para importar o conjunto de dados do PAKDD, use a função `pd.read_table` ao invés da `pd.read_csv`.

In [ ]:
data_set = pd.read_csv('data/mammography.csv.zip')
data_set.drop_duplicates(inplace=True)  # Remove exemplos repetidos

Separando o data set em atributos dependentes (X = features) e independentes (y = classe). No caso do `mammography` a classe majoritária está codificada como -1 e a classe minoritária está codificada como 1. Para treinar nossa rede neural precisamos que os valores de classe sejam 0 e 1, assim modificamos a codificação da majoritária para 0.

Perceba que esse pré-processamento varia de data set para data set.

In [ ]:
X = data_set.iloc[:, :-1].values
y = data_set.iloc[:, -1].values
y = np.where(y == -1, 0, 1)

## MLP

Aqui definimos a arquitetura de nossa rede neural e o processo de treinamento e teste da mesma.

O treinamento é feito através de repetidas divisões do conjunto de dados em treinamento, validação e teste. O número de repetições e os tamanhos dos conjuntos de treino, validação, e teste podem ser facilmente modificados.

Para testar o comportamento da rede com diferentes funções de sampling, as mesmas devem ser implementadas e aplicadas ao conjunto de treinamento antes da normalização dos dados (você também pode investigar qual o efeito de aplicar o sampling após a normalização).

É importante lembrar de normalizar os dados. A classe `StandardScaler` centraliza as variáveis e transforma as features para terem variância unitária. Você pode testar outras opções como o `MinMaxScaler`.

Todas as alternativas estão disponíveis em:
http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing.

A arquitetura da rede é então definida como tendo apenas uma camada escondida. O código é bem intuitivo e a adição de novas camadas pode ser feita através da função `add`.

Várias funções de otimização estão disponíveis e seus parâmetros também podem ser definidos. 

Confira os exemplos em: https://keras.io/optimizers/

Um maior controle sobre quando a rede deve parar de treinar também pode ser exercido. 

Confira a documentação da classe `EarlyStopping`: https://keras.io/callbacks/

Por fim, predições são feitas para o conjunto de teste e métricas de desempenho são calculadas.

Mais métricas de desempenho: http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics

In [ ]:
accuracies = []
recalls = []
precisions = []
f1s = []
aucs = []

for _ in range(10):
    ## Treino: 60%, Validação: 20%, Teste: 20%
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
    
    ## Aplicar a funcao de sampling ao conjunto de treinamento
    # TO DO
    
    ## Normalizar as features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    ## Treinar a rede
    # Cria o esboço da rede.
    classifier = Sequential()
    # Adiciona a primeira camada escondida contendo 16 neurônios e função de ativação tangente 
    # hiperbólica. Por ser a primeira camada adicionada à rede, precisamos especificar a 
    # dimensão de entrada (número de features do data set), no caso do mammography são 6.
    classifier.add(Dense(16, activation='tanh', input_dim=6))
    # Adiciona a camada de saída. Como nosso problema é binário, só precisamos de 1 neurônio 
    # e função de ativação sigmoidal. A partir da segunda camada adicionada, keras já consegue 
    # inferir o número de neurônios de entrada (nesse caso 16) e nós não precisamos mais 
    # especificar.
    classifier.add(Dense(1, activation='sigmoid'))
    # Compila o modelo especificando o otimizador, a função de custo, e opcionalmente métricas 
    # para serem observadas durante o treinamento.
    classifier.compile(optimizer='adam', loss='binary_crossentropy')
    # Treina a rede, especificando o tamanho do batch, o número máximo de épocas, se deseja 
    # parar prematuramente caso o erro de validação não decresça, e o conjunto de validação.
    classifier.fit(X_train, y_train, batch_size=32, epochs=10000, 
                   callbacks=[EarlyStopping()], validation_data=(X_val, y_val))
    
    ## Fazer predições no conjunto de teste
    y_pred = classifier.predict(X_test)
    y_pred_class = classifier.predict_classes(X_test)
    ## Computar métricas de desempenho
    accuracies.append(accuracy_score(y_test, y_pred_class))
    recalls.append(recall_score(y_test, y_pred_class))
    precisions.append(precision_score(y_test, y_pred_class))
    f1s.append(f1_score(y_test, y_pred_class))
    aucs.append(roc_auc_score(y_test, y_pred))

accuracies_mean, accuracies_std = np.mean(accuracies), np.std(accuracies)
recalls_mean, recalls_std = np.mean(recalls), np.std(recalls)
precisions_mean, precisions_std = np.mean(precisions), np.std(precisions)
f1s_mean, f1s_std = np.mean(f1s), np.std(f1s)
aucs_mean, aucs_std = np.mean(aucs), np.std(aucs)

In [ ]:
print('\t\t\tmean\t\t\tstd')
print('Accuracy\t', accuracies_mean, '\t', accuracies_std)
print('Recall\t\t', recalls_mean, '\t', recalls_std)
print('Precision\t', precisions_mean, '\t', precisions_std)
print('F1\t\t', f1s_mean, '\t', f1s_std)
print('AUC\t\t', aucs_mean, '\t', aucs_std)